In [1]:
import netsquid as ns
import numpy as np
import cmath
import random
import netsquid.components.instructions as instr
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.qprocessor import PhysicalInstruction
from netsquid.nodes.connections import Connection, DirectConnection
from netsquid.components import ClassicalChannel
from netsquid.components.models import FibreDelayModel
from netsquid.nodes import Node
from netsquid.components import QuantumChannel
from netsquid.qubits.qubitapi import create_qubits
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel,T1T2NoiseModel
from netsquid.protocols import NodeProtocol, Signals ,LocalProtocol  

In [2]:

class ClassicalBiConnection(DirectConnection):
    def __init__(self, length,name="ClassicalConnection"):
        
        super().__init__(name=name)
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])
        self.add_subcomponent(ClassicalChannel("Channel_B2A", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("B", "send")],
                              forward_output=[("A", "recv")])


class QuantumConnection(Connection):
    def __init__(self, length, prob,name="QuantumConnection"):
        super().__init__(name=name)
        self.prob = prob
#         Model = DepolarNoiseModel(depolar_rate = self.prob,time_independent=True)
        Model = DephaseNoiseModel(dephase_rate  = self.prob,time_independent=True)
        qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
                                      models={"delay_model": FibreDelayModel(), "quantum_noise_model" : Model})
        # Add channels and forward quantum channel output to external port output:
        self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        


In [3]:


def create_processor():

    def RandUnitary():
        basis_matrix = np.identity(2)
        R= np.zeros(2)
        Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-Theta)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    R = RandUnitary()
    R1 =  ns.qubits.operators.Operator("R1", R)
    INSTR_R = instr.IGate("R_gate", R1)
    # We'll give both Alice and Bob the same kind of processor
    num_qubits = 4
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(INSTR_R, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False)
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,phys_instructions=physical_instructions)
    return processor


def create_processor1(probs):

    def RandUnitary():
        basis_matrix = np.identity(2)
        R= np.zeros(2)
        Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-Theta)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    R = RandUnitary()
    R1 =  ns.qubits.operators.Operator("R1", R)
    INSTR_R = instr.IGate("R_gate", R1)
    # We'll give both Alice and Bob the same kind of processor
    num_qubits = 4
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(INSTR_R, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False)
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
    memory_noise_model = DephaseNoiseModel(dephase_rate  = probs,time_independent=True)
#     memory_noise_model = DepolarNoiseModel(depolar_rate  = probs,time_independent=True)
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,phys_instructions=physical_instructions)
    return processor

In [4]:
from netsquid.components.qprogram import QuantumProgram

class InitStateProgram(QuantumProgram):
    default_num_qubits = 4
#     def __init__(self,num_parties)
#         print(num_parties)
#         self.num_qubits_ = int(np.log2(num_parties))
    
    def program(self):
#         self.num_qubits = int(np.log2(self.num_qubits))
        q1,q2,q3,q4 = self.get_qubit_indices()
        self.apply(instr.INSTR_INIT, [q1,q2,q3,q4])
        self.apply(instr.INSTR_H,q2)
        self.apply(instr.INSTR_H,q4)
#         for i in range(self.num_qubits):
#             if i % 2 != 0:
#                 self.apply(instr.INSTR_H, qubits[i])
#                 print(f"Node 1 apply hadamard to pos {i}")
#         print(qubits)
        self.apply(instr.INSTR_CNOT, [q2, q1])
        self.apply(instr.INSTR_CNOT, [q4,q3])
        yield self.run()
        
class RandUnitary(QuantumProgram):
    def RandUnitary(self):
        basis_matrix = np.identity(2)
        R= np.zeros(2)
        Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-Theta)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    def program(self):
        R = self.RandUnitary()
        R1 =  ns.qubits.operators.Operator("R1", R)
        INSTR_R = instr.IGate("R_gate", R1)
        self.apply(INSTR_R, 0)
        yield self.run()
        
class MeasureZ(QuantumProgram):
#     default_num_qubits = 4
    def program(self,mem_pos):
        qubits = self.get_qubit_indices()
        for i in range(len(mem_pos)):
            self.apply(instr.INSTR_MEASURE,qubits[mem_pos[i]], output_key="M"+str(mem_pos[i]))
        yield self.run()
        
class MeasureX(QuantumProgram):
    def program(self,mem_pos):
        qubits = self.get_qubit_indices()
        for i in range(len(mem_pos)):
            self.apply(instr.INSTR_H, qubits[mem_pos[i]])
            self.apply(instr.INSTR_MEASURE,qubits[mem_pos[i]], output_key="M"+str(mem_pos[i]))
        yield self.run()

In [5]:
class InitSend(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 4
        #Program to initialize the qubits in the memory, input param: number of qubits
        qubit_init_program = InitStateProgram(num_qubits=qubit_number)
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)

        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])
#         print(list_classic)
#         print(list_quantum)
        
#         print(self.node.name[1])
        node_num = int(self.node.name.replace('P','')) # Current Node Number    
        
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Init qubits in memory
#             print("Loop start")
#             self.node.qmemory.peek([0,1,2,3])
            self.node.qmemory.execute_program(qubit_init_program)
            
#             print(f"Node {node_num} init qubit program")
#             yield self.await_program(self.node.qmemory)
            expr = yield (self.await_program(self.node.qmemory))
#             print(self.node.qmemory.measure())
            #Send 1 qubit to first party
            qubit1 = self.node.qmemory.pop(positions=0)
            self.node.ports[list_quantum[0]].tx_output(qubit1) 
#             print(f"Node {node_num} send qubit to Node {list_quantum[0][-1]}")
            #Send 1 qubit to second party
            qubit2 = self.node.qmemory.pop(positions=2)
            self.node.ports[list_quantum[1]].tx_output(qubit2)
            
#             print(f"Node {node_num} send qubit to Node {list_quantum[1][-1]}")
            
#           Wait for ACK
            i=0
            while (i<=self.num_nodes-2):
                if len(self.node.ports[list_classic[-1-i]].input_queue) != 0:
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                    self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                else:
#                     print(f"Node 1 waitting from node {list_classic[-1-i][-1]}")
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                     print(message)
                i = i+1
                
            #Measure qubit 
            c = random.randint(0,1)
#             c = 0
            basis_sum = c
            if c == 0:
#                 print("Node 1 measure in Z basis")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[1,3])
#                 print("Node 1 output")
#                 print(measure_program1.output)
#                 self.node.qmemory.peek([0,1,2,3])
                
            else:
#                 print("Node 1 measure in X basis")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[1,3])
#                 print("Node 1 output")
#                 print(measure_program2.output)
#                 self.node.qmemory.peek([0,1,2,3])
                
            
            
            #Wait for basis results
            i=0
            while (i<=self.num_nodes-2):
#                 print(f"Node 1 await from node {list_classic[-1-i][-1]}")
                yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                 print(message)
                basis_sum = basis_sum + message
                i = i+1
            
            #Send  basis
            self.node.ports[list_classic[0]].tx_output(c)
            self.node.ports[list_classic[1]].tx_output(c)
            
            if (basis_sum % self.num_nodes == 0):
#                 print(f"List record index {x}")
                if c == 0:
                    global_list[x][0] = 1*measure_program1.output["M3"][0] + 2*measure_program1.output["M1"][0]
                else:
                     global_list[x][0] = 1*measure_program2.output["M3"][0] + 2*measure_program2.output["M1"][0]
                x = x+1
                basis_sum = 0

class RecvMeas(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 1
        #Program to initialize the qubits in the memory, input param: number of qubits
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)
        
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])

        node_num = int(self.node.name.replace('P','')) # Current Node Number    
#         print(list_classic)
#         print(list_quantum)
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Wait for qubit
            yield self.await_port_input(self.node.ports[list_quantum[0]])
#             print(self.node.qmemory.peek([0,1,2,3]))
#             pos = list(range(0, qubit_number)
            #Send ACK
#             print(f"Node {node_num} send ACK to node {list_classic[0][-1]}")
            self.node.ports[list_classic[0]].tx_output("ACK")
#             print(f"Node {node_num} send ACK to node {list_classic[1][-1]}")
            self.node.ports[list_classic[1]].tx_output("ACK")
            
            #Wait for ACK
#             print(f"Node {node_num} wait ACK from node {list_classic[1][-1]}")
            if len(self.node.ports[list_classic[1]].input_queue) != 0:
#                 print("Queue case ACK")
                print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
                print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
            else:
#                 print("No queue case ACK")
                yield self.await_port_input(self.node.ports[list_classic[1]])
                message = self.node.ports[list_classic[1]].rx_input().items[0]
#                 print(message)
            
            #Measure qubit
            c = random.randint(0,1)
#             c = 0
#             print(c)
            if c == 0:
#                 print(f"Node {node_num} measure in Z basis ")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program1.output)
#                 self.node.qmemory.discard(0)
            else:
#                 print(f"Node {node_num} measure in X basis ")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program2.output)
#                 self.node.qmemory.discard(0)
            basis_sum = c

            i=0
            while (i<self.num_nodes-1):
#                 print(f"Node {node_num} Loop for basis announcement index: {i}")
                if (i == (self.num_nodes-node_num)):
                    for j in range(self.num_nodes-1):
                        self.node.ports[list_classic[j]].tx_output(c)
#                         print(f"Node {node_num} send basis to port {list_classic[j]}")
                        
#                 print(f"Node {node_num} wait basis from port {list_classic[-1-i]}")
                if len(self.node.ports[list_classic[1]].input_queue) != 0:
#                     print("Queue case basis")
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                    message = self.node.ports[list_classic[-1-i]].input_queue[0][1].items[0]
                    self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
#                     print(message)
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                else:
#                     print("No queue case basis")
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                     print(message)
                
#                 print(f"Node {node_num} Received basis from node {list_classic[-1-i][-1]}")
#                 print(message)
                basis_sum = basis_sum + message
                i= i+1
             #Send  basis
#             self.node.ports[list_classic[0]].tx_output(c)
#             self.node.ports[list_classic[1]].tx_output(c)
            
            #Record basis
            if (basis_sum % self.num_nodes == 0):
                if c == 0:
                    global_list[x][node_num-1] = measure_program1.output["M0"][0]
                else:
                     global_list[x][node_num-1] = measure_program2.output["M0"][0]
                basis_sum = 0
                x = x+1
                if (x > self.list_length-1):
                    if node_num == 3:
#                         print(f"List distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
#             #Send measurement results
#             yield self.await_port_input(self.node.ports[list_classic[0]])
#             message = self.node.ports[list_classic[0]].rx_input().items[0]
#             print(message)
            
class RecvMeas1(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 1
        #Program to initialize the qubits in the memory, input param: number of qubits
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)
        randU_program = RandUnitary()
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])

        node_num = int(self.node.name.replace('P','')) # Current Node Number    
#         print(list_classic)
#         print(list_quantum)
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Wait for qubit
            yield self.await_port_input(self.node.ports[list_quantum[0]])
#             print(self.node.qmemory.peek([0,1,2,3]))
#             pos = list(range(0, qubit_number)
            #Send ACK
#             print(f"Node {node_num} send ACK to node {list_classic[0][-1]}")
            self.node.ports[list_classic[0]].tx_output("ACK")
#             print(f"Node {node_num} send ACK to node {list_classic[1][-1]}")
            self.node.ports[list_classic[1]].tx_output("ACK")
            
            #Wait for ACK
#             print(f"Node {node_num} wait ACK from node {list_classic[1][-1]}")
            if len(self.node.ports[list_classic[1]].input_queue) != 0:
#                 print("Queue case ACK")
                print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
                print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
            else:
#                 print("No queue case ACK")
                yield self.await_port_input(self.node.ports[list_classic[1]])
                message = self.node.ports[list_classic[1]].rx_input().items[0]
#                 print(message)
            yield self.node.qmemory.execute_program(randU_program)
            #Measure qubit
            c = random.randint(0,1)
#             c = 0
#             print(c)
            if c == 0:
#                 print(f"Node {node_num} measure in Z basis ")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program1.output)
#                 self.node.qmemory.discard(0)
            else:
#                 print(f"Node {node_num} measure in X basis ")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program2.output)
#                 self.node.qmemory.discard(0)
            basis_sum = c

            i=0
            while (i<self.num_nodes-1):
#                 print(f"Node {node_num} Loop for basis announcement index: {i}")
                if (i == (self.num_nodes-node_num)):
                    for j in range(self.num_nodes-1):
                        self.node.ports[list_classic[j]].tx_output(c)
#                         print(f"Node {node_num} send basis to port {list_classic[j]}")
                        
#                 print(f"Node {node_num} wait basis from port {list_classic[-1-i]}")
                if len(self.node.ports[list_classic[1]].input_queue) != 0:
#                     print("Queue case basis")
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                    message = self.node.ports[list_classic[-1-i]].input_queue[0][1].items[0]
                    self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
#                     print(message)
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                else:
#                     print("No queue case basis")
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                     print(message)
                
#                 print(f"Node {node_num} Received basis from node {list_classic[-1-i][-1]}")
#                 print(message)
                basis_sum = basis_sum + message
                i= i+1
             #Send  basis
#             self.node.ports[list_classic[0]].tx_output(c)
#             self.node.ports[list_classic[1]].tx_output(c)
            
            #Record basis
            if (basis_sum % self.num_nodes == 0):
                if c == 0:
                    global_list[x][node_num-1] = measure_program1.output["M0"][0]
                else:
                     global_list[x][node_num-1] = measure_program2.output["M0"][0]
                basis_sum = 0
                x = x+1
                if (x > self.list_length-1):
                    if node_num == 3:
#                         print(f"List distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
#             #Send measurement results
#             yield self.await_port_input(self.node.ports[list_classic[0]])
#             message = self.node.ports[list_classic[0]].rx_input().items[0]
#             print(message)
                
                

In [6]:
from netsquid.nodes import Network
def example_network_setup(num_nodes,prob,node_distance=4e-3):
#     print("Network Setup")
    nodes =[]
    i = 1
    while i<=(num_nodes):
        if i == 0:
#             nodes.append(Node(f"P{i}",qmemory = create_processor1(prob)))
            nodes.append(Node(f"P{i}",qmemory = create_processor()))
        else:
            nodes.append(Node(f"P{i}",qmemory = create_processor()))
        
        i= i+1
    
    # Create a network
    network = Network("List Distribution Network")
#     print(nodes)
    network.add_nodes(nodes)
#     print("Nodes completed")
    i = 1
    while i< (num_nodes):
        node = nodes[i-1]
        j = 1
        while j<=(num_nodes-i):
            node_next = nodes[i+j-1]
            c_conn = ClassicalBiConnection(name =f"c_conn{i}{i+j}", length = node_distance)
            network.add_connection(node,node_next, connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{i}{i+j}", port_name_node2 = f"cio_node_port{i+j}{i}")
            j = j+1
        i = i+1
#     prob = 0
#     print("Classical Conn Completed")
    q_conn = QuantumConnection(name=f"qconn_{1}{2}", length=node_distance,prob=prob)
    network.add_connection(nodes[0], nodes[1], connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{1}{2}", port_name_node2=f"qin_node_port{2}{1}")
    q_conn = QuantumConnection(name=f"qconn_{1}{3}", length=node_distance,prob=prob)
    network.add_connection(nodes[0], nodes[2], connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{1}{3}", port_name_node2=f"qin_node_port{3}{1}")
    


    i = 2
    while i<=(num_nodes):
        nodes[i-1].ports[f"qin_node_port{i}{1}"].forward_input(nodes[i-1].qmemory.ports['qin'])
        i = i+1
#     print("End Network Setup")
    
    return network

In [7]:
def setup_protocol(network,nodes_num,fault_num,list_length):
#     print("Setup Protocol")
    
    protocol = LocalProtocol(nodes=network.nodes)
    nodes = []
    i = 1
    while i<=(nodes_num):
        nodes.append(network.get_node("P"+str(i)))
        i = i+1
#     print(nodes)

    
    subprotocol = InitSend(node=nodes[0],name=f"Init_Send{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
#     subprotocol = FaultyInitSend(node=nodes[0],name=f"Faulty Init_Send{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    
    #Uncomment first line for normal phase reference, uncomment second line for phase reference error
    subprotocol = RecvMeas(node=nodes[1], name=f"Receive_Measure{nodes[1].name}",num_nodes=nodes_num,list_length=list_length)
#     subprotocol = RecvMeas1(node=nodes[1], name=f"Receive_Measure{nodes[1].name}",num_nodes=nodes_num,list_length=list_length)
    
    protocol.add_subprotocol(subprotocol)
    subprotocol = RecvMeas(node=nodes[2], name=f"Receive_Measure{nodes[2].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    return protocol


In [9]:
# Time Data Collection


from netsquid.util.simtools import set_random_state
import pandas as pd

# set up initial parameters
nodes_num = 3 #Node numbers

fault_num = 0 #Faulty node numbers
exp_number = 1 #Experiment numbers
probs = np.linspace(0, 1, num=100)
# print(probs)
exp_number = len(probs)
list_length = 100 #List length
error_array = np.ndarray(shape=(exp_number,2))
time_array = np.ndarray(shape=(exp_number,))
global_list = np.ndarray(shape=(list_length,nodes_num), dtype='i')
x=0
ns.sim_reset()
network = example_network_setup(nodes_num,0,node_distance=4)
protocol = setup_protocol(network,nodes_num,fault_num,list_length)
protocol.start()
stats = ns.sim_run()
# protocol = setup_protocol(network,nodes_num,fault_num,list_length)
average = 100
while x < len(probs):

    global_list = np.ndarray(shape=(list_length,nodes_num), dtype='i')
    error_sum = 0
    for z in range (average):
        ns.sim_reset()
        network = example_network_setup(nodes_num,probs[x],node_distance=4)
        protocol = setup_protocol(network,nodes_num,fault_num,list_length)
        protocol.start()
        stats = ns.sim_run()

        if (ns.sim_state() == 2):
#             print(f"Sim end time: {ns.sim_time()}")
            valid_sum = 0
            for i in range(global_list.shape[0]-1):
                if ((global_list[i][0] == (global_list[i][1]*2+global_list[i][2]))):
                    valid_sum = valid_sum+1
    #         percentage_correct = (valid_sum/(global_list.shape[0]-1)) * 100
            percentage_correct = (valid_sum/(global_list.shape[0]-1))
            error_ratio = 1-percentage_correct
            error_sum = error_sum + error_ratio
        
    print(f"Error Prob = {probs[x]} Average Ratio of Error List:{error_sum/average}")
    error_array[x][0] = probs[x]
    error_array[x][1] = error_sum/average
    x = x+1



Error Prob = 0.0 Average Ratio of Error List:0.0
Error Prob = 0.010101010101010102 Average Ratio of Error List:0.008383838383838372
Error Prob = 0.020202020202020204 Average Ratio of Error List:0.02151515151515149
Error Prob = 0.030303030303030304 Average Ratio of Error List:0.026363636363636356
Error Prob = 0.04040404040404041 Average Ratio of Error List:0.04040404040404041
Error Prob = 0.05050505050505051 Average Ratio of Error List:0.04858585858585859
Error Prob = 0.06060606060606061 Average Ratio of Error List:0.057373737373737355
Error Prob = 0.07070707070707072 Average Ratio of Error List:0.06838383838383837
Error Prob = 0.08080808080808081 Average Ratio of Error List:0.08141414141414138
Error Prob = 0.09090909090909091 Average Ratio of Error List:0.08939393939393944
Error Prob = 0.10101010101010102 Average Ratio of Error List:0.09222222222222219
Error Prob = 0.11111111111111112 Average Ratio of Error List:0.1015151515151515
Error Prob = 0.12121212121212122 Average Ratio of Error

In [10]:
# print(global_list)
# print(error_array)
error_data = pd.DataFrame(data = error_array,columns = ['error probability','Error List Ratio'])
print(error_data)
error_data.to_csv('QKD_DephaseChannel_100avrg.csv')

    error probability  Error List Ratio
0            0.000000          0.000000
1            0.010101          0.008384
2            0.020202          0.021515
3            0.030303          0.026364
4            0.040404          0.040404
..                ...               ...
95           0.959596          0.498384
96           0.969697          0.500808
97           0.979798          0.498384
98           0.989899          0.490303
99           1.000000          0.499091

[100 rows x 2 columns]
